# AutoGen Check

In [1]:
%pip install "autogen-ext[openai,azure]"
%pip install azure-identity
%pip install autogen_ext


  Using cached distro-1.9.0-py3-none-any.whl.metadata (6.8 kB)
  Using cached httpx-0.28.1-py3-none-any.whl.metadata (7.1 kB)
  Using cached sniffio-1.3.1-py3-none-any.whl.metadata (3.9 kB)
  Using cached httpcore-1.0.7-py3-none-any.whl.metadata (21 kB)
  Using cached h11-0.14.0-py3-none-any.whl.metadata (8.2 kB)
  Using cached annotated_types-0.7.0-py3-none-any.whl.metadata (15 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 643.9/643.9 kB 18.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 40.2 MB/s eta 0:00:00
Using cached distro-1.9.0-py3-none-any.whl (20 kB)
Using cached httpx-0.28.1-py3-none-any.whl (73 kB)
Using cached httpcore-1.0.7-py3-none-any.whl (78 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.5/4.5 MB 95.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 61.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 781.7/781.7 kB 28.3 MB/s eta 0:00:00
Using cached sniffio-1.3.1-py3-none-any.whl (10 kB)
Using cached a

In [2]:
import json
with open("Dataset/user_instructions.json", 'r') as f:
    user_profiles = json.load(f)
print("Number of users loaded:",len(user_profiles))

#find length of user_profiles test and train
print("Number of users in train:",len(user_profiles['train']))  
print("Number of users in test:",len(user_profiles['test']))

# merge user_profiles['train'] and user_profiles['test']
user_profiles['train'] += user_profiles['test']

#find length of user_profiles['train']
print("Final Number of users: ",len(user_profiles['train']))



Number of users loaded: 2
Number of users in train: 6896
Number of users in test: 2174
Final Number of users:  9070


In [3]:
import os
from azure.ai.inference import ChatCompletionsClient
from azure.ai.inference.models import AssistantMessage, SystemMessage, UserMessage
from azure.core.credentials import AzureKeyCredential
from autogen import ConversableAgent
import sys
import json
import numpy as np
import pandas as pd



In [ ]:
os.environ["AZURE_OPENAI_API_KEY"] = "YOUR API"
os.environ["AZURE_OPENAI_API_BASE"] = "API BASE"
# Define the llm_config
config_list={
        "model": "gpt-4o-mini",
        "api_key": os.environ.get("AZURE_OPENAI_API_KEY"),
        "api_type": "azure",
        "base_url": os.environ.get("AZURE_OPENAI_API_BASE"),
        "api_version": "2025-01-01-preview",
    }


In [12]:
import json

# Load JSON file
with open("Dataset/user_profiles.json", "r", encoding="utf-8") as file:
    personas = json.load(file)

def generate_prompt(user_profile):
    gender = user_profile["user_profile"].get("Gender", "Unknown").lower()
    pronoun = "they" if gender not in ["male", "female"] else "she" if gender == "female" else "he"
    
    return f"""
You are a shopper looking to purchase an item. Stay in character as a {user_profile["user_profile"].get('Age', 'unknown age')}-year-old {gender} {user_profile["user_profile"].get('Occupation', 'individual')}.
Your goal is to find the best option based on your preferences, price sensitivity ({user_profile["user_profile"].get('Price Sensitivity', 'unknown')}), and shopping interests ({user_profile["user_profile"].get('Shopping Interest', 'various categories')}).
When responding:
- Provide information about your preferences, interests, and constraints.
- Ask questions about product details, discounts, or comparisons.
- Express opinions and react naturally to recommendations.
- Do **not** provide recommendations or act like an assistant.
- Keep your tone {user_profile["user_profile"].get('Tone and Style', 'neutral')}.
- If unsure, ask for clarification rather than making recommendations.
"""

# Process all personas
for user in personas:
    persona = personas[user]
    sys_message = generate_prompt(persona)

    user_proxy = ConversableAgent(
        "user",
        system_message = sys_message,
        llm_config = config_list,
        human_input_mode = "NEVER",
        description="You are a shopper trying to present their preferences and constraints to a shopping assistant.",
    )

    shopping_assistant = ConversableAgent(
        "assistant",
        system_message=(
            "You are a highly capable shopping assistant designed to help users with their shopping needs. "
            "Your primary work task is to assist end users in their shopping journey by handling three core subtasks: "
            "1. Product Discovery: Identify and suggest relevant products based on user preferences. "
            "2. Price Comparison: Provide competitive price analysis across multiple vendors. "
            "3. Personalized Recommendations: Tailor suggestions based on user history, preferences, and context. "
            "\nTo achieve these tasks, you must conduct a structured reference interview to elicit user needs, including their prefernces "
            "like preferred brands, price range, product features, shopping urgency, and other relevant factors. "
            "Your goal is to efficiently gather *** yser's needs, requirements and preferences*** without overwhelming the user. Ask few questions at a time. "
        ),
        llm_config=config_list,
        human_input_mode="NEVER",  # Never ask for human input.
    )
    # For every user in personas, check if the user is in user_profiles['train']
    print(f"Checking user: {user}")
    
    # Look for user in user_profiles['train'] where user is a key named as user_id
    found = any(profile.get('user_id') == user for profile in user_profiles['train'])
    #extract 'task' corresponding to user in user_profiles['train']
    task = [profile['task'] for profile in user_profiles['train'] if profile.get('user_id') == user]
    print("Task:",task[0])
    print("Task1:",task[1])
    
    if found:
        print("User found in train")
    else:
        print("User not found in train")
    user_query = task[0]
    result = user_proxy.initiate_chat(shopping_assistant, message= user_query , max_turns=5)

    # Save chat history in a text file
    with open(f"Results_referenceInterview/{user}.txt", 'w') as f_txt:
        for message in result.chat_history:
            f_txt.write(f"{message['role'].capitalize()}: {message['content']}\n\n")



Checking user: AGMQSZEQFFKH33FJQZLN7MF5QX2Q
Task: Hey there! I'm looking for a high-quality EMF protection sleeve for my phone. Something reliable from a reputable brand that fits easily in pockets. Any suggestions?
Task1: Hey there! I'm really impressed with these Halsa EMF protection sleeves. Could you help me write a review that highlights the quality, effectiveness, and practicality? I'd like to mention specific features and compare them to other brands I've tried. Thanks!
User found in train
user (to assistant):

Hey there! I'm looking for a high-quality EMF protection sleeve for my phone. Something reliable from a reputable brand that fits easily in pockets. Any suggestions?

--------------------------------------------------------------------------------
assistant (to user):

Great! I can help you find the right EMF protection sleeve for your phone. To get started, could you share a few more details?

1. What specific phone model do you have, as that will help ensure a proper fi

KeyboardInterrupt: 